<a href="https://colab.research.google.com/github/armandossrecife/mysentimentanalysis/blob/main/my_automatic_inspection_issues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get my dataset

In [ ]:
!wget https://raw.githubusercontent.com/armandossrecife/mychatbot/issue_7/issues_ground_truth.json

--2024-07-06 20:57:45--  https://raw.githubusercontent.com/armandossrecife/mychatbot/issue_7/issues_ground_truth.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1593692 (1.5M) [text/plain]
Saving to: ‘issues_ground_truth.json’

issues_ground_truth 100%[===================>]   1.52M  --.-KB/s    in 0.04s   

2024-07-06 20:57:46 (42.3 MB/s) - ‘issues_ground_truth.json’ saved [1593692/1593692]



In [ ]:
!wget https://raw.githubusercontent.com/Technical-Debt-Large-Scale/my_validation/main/cassandra_issues_inspected_merged.xlsx

--2024-07-06 20:57:46--  https://raw.githubusercontent.com/Technical-Debt-Large-Scale/my_validation/main/cassandra_issues_inspected_merged.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 573631 (560K) [application/octet-stream]
Saving to: ‘cassandra_issues_inspected_merged.xlsx’

cassandra_issues_in 100%[===================>] 560.19K  --.-KB/s    in 0.03s   

2024-07-06 20:57:46 (17.0 MB/s) - ‘cassandra_issues_inspected_merged.xlsx’ saved [573631/573631]



# Install dependencies

In [ ]:
!pip install datasets

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate -U

# Import libs

In [ ]:
import pandas as pd
import json
from datasets import load_dataset
import json
import numpy as np

# My Functions

In [ ]:
def convert_comment_to_text(comments_string):
  try:
    comments_string = comments_string.split(": ")[-1]  # Assuming the colon and space separate the key-value pair
    comments_list = comments_string.split(",")
    comments = ""
    for comment in comments_list:
        comment = comment.replace("['", "")
        comment = comment.replace("']", "")
        comment = comment.strip()
        comments += comment + " "
    return comments
  except:
    return ""

def convert_inspection_to_binary(inspection):
  if inspection == 'YES':
    return 1
  else:
    return 0

def convert_inspection_to_sentiment(inspection):
  if inspection == 'YES':
    return 'positive'
  else:
    return 'negative'

def convert_to_json(df):
    try:
      json_data = df.to_json(orient='records')
      return json.loads(json_data)
    except Exception as e:
      raise ValueError(e)

def write_json_to_file(json_data, filename):
    try:
      with open(filename, 'w') as f:
        json.dump(json_data, f)
    except Exception as e:
      raise ValueError(e)

def convert_dataframe_to_json_file(df, filename):
  try:
    json_string_data = convert_to_json(df)
    write_json_to_file(json_string_data, filename)
  except Exception as e:
    raise ValueError(e)

# DataFrame Cleaner

In [ ]:
with open('issues_ground_truth.json') as f:
    data = json.load(f)

df = pd.json_normalize(data)
df_issues_ground_truth = pd.DataFrame(df)
df_issues_ground_truth.head(3)

,issue_key,issue_type,summary,description,comments,architectural_impact_manual
0,CASSANDRA-11272,Bug,NullPointerException (NPE) during bootstrap st...,After bootstrapping fails due to stream closed...,"['Which Cassandra version are you using? ', 'T...",YES
1,CASSANDRA-1321,Improvement,loadSchemaFromYaml should push migrations to c...,None,"['announces the last migration, which ends up ...",YES
2,CASSANDRA-1641,Bug,auto-guessed memtable sizes are too high,I've seen two cases now of the memtable sizes ...,"[""I'd like to introduce a dependency on number...",YES


In [ ]:
df_cassandra_issues_inspected_merged = pd.read_excel('cassandra_issues_inspected_merged.xlsx')
df_cassandra_issues_inspected_merged.head(3)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact_manual,architectural_impact,TP,TN,FP,FN
0,CASSANDRA-11272,NullPointerException (NPE) during bootstrap st...,Bug,Resolved,Normal,After bootstrapping fails due to stream closed...,"['Which Cassandra version are you using? ', 'T...",YES,YES,1,0,0,0
1,CASSANDRA-1321,loadSchemaFromYaml should push migrations to c...,Improvement,Resolved,Normal,NaN,"['announces the last migration, which ends up ...",YES,YES,1,0,0,0
2,CASSANDRA-1641,auto-guessed memtable sizes are too high,Bug,Resolved,Normal,I've seen two cases now of the memtable sizes ...,"[""I'd like to introduce a dependency on number...",YES,YES,1,0,0,0


## Dados da Inspeção Manual

In [ ]:
colunas_manual = ['issue_key', 'summary', 'issue_type', 'issue_status', 'issue_priority',
       'description', 'comments', 'architectural_impact_manual']

df_cassandra_issues_inspected_manual = df_cassandra_issues_inspected_merged[colunas_manual]
df_cassandra_issues_inspected_manual.head(3)

,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact_manual
0,CASSANDRA-11272,NullPointerException (NPE) during bootstrap st...,Bug,Resolved,Normal,After bootstrapping fails due to stream closed...,"['Which Cassandra version are you using? ', 'T...",YES
1,CASSANDRA-1321,loadSchemaFromYaml should push migrations to c...,Improvement,Resolved,Normal,NaN,"['announces the last migration, which ends up ...",YES
2,CASSANDRA-1641,auto-guessed memtable sizes are too high,Bug,Resolved,Normal,I've seen two cases now of the memtable sizes ...,"[""I'd like to introduce a dependency on number...",YES


In [ ]:
df_cassandra_issues_inspected_manual['comments_text'] = df_cassandra_issues_inspected_manual['comments'].apply(convert_comment_to_text)
df_cassandra_issues_inspected_manual['label'] = df_cassandra_issues_inspected_manual['architectural_impact_manual'].apply(convert_inspection_to_binary)
df_cassandra_issues_inspected_manual['label_text'] = df_cassandra_issues_inspected_manual['architectural_impact_manual'].apply(convert_inspection_to_sentiment)

<ipython-input-142-0460dc42e635>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cassandra_issues_inspected_manual['comments_text'] = df_cassandra_issues_inspected_manual['comments'].apply(convert_comment_to_text)
<ipython-input-142-0460dc42e635>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cassandra_issues_inspected_manual['label'] = df_cassandra_issues_inspected_manual['architectural_impact_manual'].apply(convert_inspection_to_binary)
<ipython-input-142-0460dc42e635>:3: SettingWithCopyWarning: 

In [ ]:
df_cassandra_issues_inspected_manual.head(3)

,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact_manual,comments_text,label,label_text
0,CASSANDRA-11272,NullPointerException (NPE) during bootstrap st...,Bug,Resolved,Normal,After bootstrapping fails due to stream closed...,"['Which Cassandra version are you using? ', 'T...",YES,patch is at the end of [ifesdjeen/11272-2.2|ht...,1,positive
1,CASSANDRA-1321,loadSchemaFromYaml should push migrations to c...,Improvement,Resolved,Normal,NaN,"['announces the last migration, which ends up ...",YES,announces the last migration which ends up pus...,1,positive
2,CASSANDRA-1641,auto-guessed memtable sizes are too high,Bug,Resolved,Normal,I've seen two cases now of the memtable sizes ...,"[""I'd like to introduce a dependency on number...",YES,"[""I'd like to introduce a dependency on number...",1,positive


## Dados da Inspeção Semi-Automática (via ChatGPT)

In [ ]:
colunas_automatica = ['issue_key', 'summary', 'issue_type', 'issue_status', 'issue_priority',
       'description', 'comments', 'architectural_impact']

df_cassandra_issues_inspected_automatica = df_cassandra_issues_inspected_merged[colunas_automatica]
df_cassandra_issues_inspected_automatica.head(3)

,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact
0,CASSANDRA-11272,NullPointerException (NPE) during bootstrap st...,Bug,Resolved,Normal,After bootstrapping fails due to stream closed...,"['Which Cassandra version are you using? ', 'T...",YES
1,CASSANDRA-1321,loadSchemaFromYaml should push migrations to c...,Improvement,Resolved,Normal,NaN,"['announces the last migration, which ends up ...",YES
2,CASSANDRA-1641,auto-guessed memtable sizes are too high,Bug,Resolved,Normal,I've seen two cases now of the memtable sizes ...,"[""I'd like to introduce a dependency on number...",YES


In [ ]:
df_cassandra_issues_inspected_automatica['comments_text'] = df_cassandra_issues_inspected_automatica['comments'].apply(convert_comment_to_text)
df_cassandra_issues_inspected_automatica['label'] = df_cassandra_issues_inspected_automatica['architectural_impact'].apply(convert_inspection_to_binary)
df_cassandra_issues_inspected_automatica['label_text'] = df_cassandra_issues_inspected_automatica['architectural_impact'].apply(convert_inspection_to_sentiment)

<ipython-input-145-5f27aec7d710>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cassandra_issues_inspected_automatica['comments_text'] = df_cassandra_issues_inspected_automatica['comments'].apply(convert_comment_to_text)
<ipython-input-145-5f27aec7d710>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cassandra_issues_inspected_automatica['label'] = df_cassandra_issues_inspected_automatica['architectural_impact'].apply(convert_inspection_to_binary)
<ipython-input-145-5f27aec7d710>:3: SettingWithCopy

In [ ]:
df_cassandra_issues_inspected_automatica.head()

,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact,comments_text,label,label_text
0,CASSANDRA-11272,NullPointerException (NPE) during bootstrap st...,Bug,Resolved,Normal,After bootstrapping fails due to stream closed...,"['Which Cassandra version are you using? ', 'T...",YES,patch is at the end of [ifesdjeen/11272-2.2|ht...,1,positive
1,CASSANDRA-1321,loadSchemaFromYaml should push migrations to c...,Improvement,Resolved,Normal,NaN,"['announces the last migration, which ends up ...",YES,announces the last migration which ends up pus...,1,positive
2,CASSANDRA-1641,auto-guessed memtable sizes are too high,Bug,Resolved,Normal,I've seen two cases now of the memtable sizes ...,"[""I'd like to introduce a dependency on number...",YES,"[""I'd like to introduce a dependency on number...",1,positive
3,CASSANDRA-2296,"Scrub resulting in ""bloom filter claims to be ...",Bug,Resolved,Normal,Doing a scrub on a node which I upgraded from ...,"[""With debug logging turned on it looks like t...",YES,Keys must be written in ascending order.\n ...,1,positive
4,CASSANDRA-3117,StorageServiceMBean is missing a getCompaction...,Bug,Resolved,Low,"Without a getter, you can assign a new value b...","['+1', 'committed.', 'Integrated in Cassandra-...",YES,\n* /cassandra/branches/cassandra-0.8/src/java...,1,positive


# Dataset to Model

## Dataset to Train

In [ ]:
df_cassandra_issues_inspected_manual['architectural_impact'] = df_cassandra_issues_inspected_manual['architectural_impact_manual']

<ipython-input-147-4577b7183cee>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cassandra_issues_inspected_manual['architectural_impact'] = df_cassandra_issues_inspected_manual['architectural_impact_manual']


In [ ]:
my_columns = ['issue_key', 'summary', 'issue_type', 'issue_status', 'issue_priority',
       'description', 'comments', 'architectural_impact', 'comments_text',
       'label', 'label_text']

df_cassandra_issues_inspected_manual = df_cassandra_issues_inspected_manual[my_columns]
df_cassandra_issues_inspected_manual.head(3)

,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact,comments_text,label,label_text
0,CASSANDRA-11272,NullPointerException (NPE) during bootstrap st...,Bug,Resolved,Normal,After bootstrapping fails due to stream closed...,"['Which Cassandra version are you using? ', 'T...",YES,patch is at the end of [ifesdjeen/11272-2.2|ht...,1,positive
1,CASSANDRA-1321,loadSchemaFromYaml should push migrations to c...,Improvement,Resolved,Normal,NaN,"['announces the last migration, which ends up ...",YES,announces the last migration which ends up pus...,1,positive
2,CASSANDRA-1641,auto-guessed memtable sizes are too high,Bug,Resolved,Normal,I've seen two cases now of the memtable sizes ...,"[""I'd like to introduce a dependency on number...",YES,"[""I'd like to introduce a dependency on number...",1,positive


In [ ]:
df_data_train = df_cassandra_issues_inspected_manual
df_data_train['description'].fillna('There is no description', inplace=True)

df_data_train.head()

<ipython-input-149-2d7218bf1509>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_train['description'].fillna('There is no description', inplace=True)


,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact,comments_text,label,label_text
0,CASSANDRA-11272,NullPointerException (NPE) during bootstrap st...,Bug,Resolved,Normal,After bootstrapping fails due to stream closed...,"['Which Cassandra version are you using? ', 'T...",YES,patch is at the end of [ifesdjeen/11272-2.2|ht...,1,positive
1,CASSANDRA-1321,loadSchemaFromYaml should push migrations to c...,Improvement,Resolved,Normal,There is no description,"['announces the last migration, which ends up ...",YES,announces the last migration which ends up pus...,1,positive
2,CASSANDRA-1641,auto-guessed memtable sizes are too high,Bug,Resolved,Normal,I've seen two cases now of the memtable sizes ...,"[""I'd like to introduce a dependency on number...",YES,"[""I'd like to introduce a dependency on number...",1,positive
3,CASSANDRA-2296,"Scrub resulting in ""bloom filter claims to be ...",Bug,Resolved,Normal,Doing a scrub on a node which I upgraded from ...,"[""With debug logging turned on it looks like t...",YES,Keys must be written in ascending order.\n ...,1,positive
4,CASSANDRA-3117,StorageServiceMBean is missing a getCompaction...,Bug,Resolved,Low,"Without a getter, you can assign a new value b...","['+1', 'committed.', 'Integrated in Cassandra-...",YES,\n* /cassandra/branches/cassandra-0.8/src/java...,1,positive


## Dataset to Test

In [ ]:
df_data_test = df_cassandra_issues_inspected_automatica
df_data_test['description'].fillna('There is no description', inplace=True)

df_data_test.head()

<ipython-input-150-b24372c1b673>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_test['description'].fillna('There is no description', inplace=True)


,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact,comments_text,label,label_text
0,CASSANDRA-11272,NullPointerException (NPE) during bootstrap st...,Bug,Resolved,Normal,After bootstrapping fails due to stream closed...,"['Which Cassandra version are you using? ', 'T...",YES,patch is at the end of [ifesdjeen/11272-2.2|ht...,1,positive
1,CASSANDRA-1321,loadSchemaFromYaml should push migrations to c...,Improvement,Resolved,Normal,There is no description,"['announces the last migration, which ends up ...",YES,announces the last migration which ends up pus...,1,positive
2,CASSANDRA-1641,auto-guessed memtable sizes are too high,Bug,Resolved,Normal,I've seen two cases now of the memtable sizes ...,"[""I'd like to introduce a dependency on number...",YES,"[""I'd like to introduce a dependency on number...",1,positive
3,CASSANDRA-2296,"Scrub resulting in ""bloom filter claims to be ...",Bug,Resolved,Normal,Doing a scrub on a node which I upgraded from ...,"[""With debug logging turned on it looks like t...",YES,Keys must be written in ascending order.\n ...,1,positive
4,CASSANDRA-3117,StorageServiceMBean is missing a getCompaction...,Bug,Resolved,Low,"Without a getter, you can assign a new value b...","['+1', 'committed.', 'Integrated in Cassandra-...",YES,\n* /cassandra/branches/cassandra-0.8/src/java...,1,positive


# Convert Dataframes do JSON

In [ ]:
# Convert the DataFrame to a JSON string
print("Convert the DataFrame to a JSON string")
convert_dataframe_to_json_file(df=df_data_train, filename='train.json')
convert_dataframe_to_json_file(df=df_data_test, filename='test.json')
print("Conversão realizada com sucesso!")

Convert the DataFrame to a JSON string
Conversão realizada com sucesso!


# Create the Model based on Facing Hugg Bert

In [ ]:
from transformers import DistilBertForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer  # Import from transformers
import torch

In [ ]:
# Define model name and task
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
task = "sentiment-analysis"

In [ ]:
# Load pre-trained DistilBERT model and tokenizer
model = DistilBertForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
MY_HUGGING_FACE_DATASET = "armandoufpi/cassandraissuesgroundtruth"

In [ ]:
# Load the IMDB movie review dataset
train_data = load_dataset(MY_HUGGING_FACE_DATASET, split="train")
test_data = load_dataset(MY_HUGGING_FACE_DATASET, split="test")

Generating train split:   0%|          | 0/226 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/226 [00:00<?, ? examples/s]

In [ ]:
train_data

Dataset({
    features: ['label_text', 'comments_text', 'issue_status', 'issue_key', 'issue_priority', 'summary', 'description', 'comments', 'architectural_impact', 'issue_type', 'label'],
    num_rows: 226
})

In [ ]:
len(train_data['summary'])

226

In [ ]:
train_data['summary'][0]

'NullPointerException (NPE) during bootstrap startup in StorageService.java'

In [ ]:
train_data['label'][0]

1

In [ ]:
train_data['label_text'][0]

'positive'

In [ ]:
train_data['description'][0]

'After bootstrapping fails due to stream closed error, the following error results:\n\n{code}\nFeb 27, 2016 8:06:38 PM com.google.common.util.concurrent.ExecutionList executeListener\nSEVERE: RuntimeException while executing runnable com.google.common.util.concurrent.Futures$6@3d61813b with executor INSTANCE\njava.lang.NullPointerException\n        at org.apache.cassandra.service.StorageService$2.onFailure(StorageService.java:1284)\n        at com.google.common.util.concurrent.Futures$6.run(Futures.java:1310)\n        at com.google.common.util.concurrent.MoreExecutors$DirectExecutor.execute(MoreExecutors.java:457)\n        at com.google.common.util.concurrent.ExecutionList.executeListener(ExecutionList.java:156)\n        at com.google.common.util.concurrent.ExecutionList.execute(ExecutionList.java:145)\n        at com.google.common.util.concurrent.AbstractFuture.setException(AbstractFuture.java:202)\n        at org.apache.cassandra.streaming.StreamResultFuture.maybeComplete(StreamResul

In [ ]:
# Function to preprocess text data
def preprocess_function_description(examples):
  return tokenizer(examples["description"], padding="max_length", truncation=True)

# Function to preprocess text data
def preprocess_function(examples):
  return tokenizer(examples["summary"], padding="max_length", truncation=True)

In [ ]:
# Preprocess train and test data
train_data = train_data.map(preprocess_function_description, batched=True)
test_data = test_data.map(preprocess_function_description, batched=True)

Map:   0%|          | 0/226 [00:00<?, ? examples/s]

Map:   0%|          | 0/226 [00:00<?, ? examples/s]

In [ ]:
# Access the 'input_ids' from the preprocessed data
#train_inputs = train_data["input_ids"]
#test_inputs = test_data["input_ids"]

In [ ]:
!rm -rf results

In [ ]:
!mkdir results

In [ ]:
!ls -l

total 5988
-rw-r--r-- 1 root root  573631 Jul  6 19:50 cassandra_issues_inspected_merged.xlsx
-rw-r--r-- 1 root root 1593692 Jul  6 19:50 issues_ground_truth.json
drwxr-xr-x 2 root root    4096 Jul  6 20:52 results
drwxr-xr-x 1 root root    4096 Jul  3 13:21 sample_data
-rw-r--r-- 1 root root 1971990 Jul  6 19:59 test.json
-rw-r--r-- 1 root root 1971970 Jul  6 19:59 train.json


In [ ]:
training_args = TrainingArguments(
    output_dir="results",  # Fixed typo (removed extra space)
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,  # Assuming you meant "size" here
    learning_rate=2e-5,
)

In [ ]:
# Create trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics="accuracy",
)

In [ ]:
# Train the model
trainer.train()

Step,Training Loss


TrainOutput(global_step=45, training_loss=0.8041855706108941, metrics={'train_runtime': 33.5232, 'train_samples_per_second': 20.225, 'train_steps_per_second': 1.342, 'total_flos': 89812896288768.0, 'train_loss': 0.8041855706108941, 'epoch': 3.0})

In [ ]:
def analyse_issue(issue_field):
  inputs = tokenizer(issue_field, padding="max_length", truncation=True, return_tensors="pt")

  # Move the model and input to GPU if available
  if torch.cuda.is_available():
    model.to('cuda')
    inputs.to('cuda')

  with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

  # Print the predicted sentiment
  if predictions == 1:
    return "Architectural Impact: Yes"
  else:
    return "Architectural Impact: No"

In [ ]:
description_issue1 = "I've seen two cases now of the memtable sizes being too large, causing OOMing.  Too-small memtables hurt performance, but too-large hurts worse when you start GC storming."
print(f"Description: {description_issue1}")
print(analyse_issue(issue_field=description_issue1))


Description: I've seen two cases now of the memtable sizes being too large, causing OOMing.  Too-small memtables hurt performance, but too-large hurts worse when you start GC storming.
Architectural Impact: No


In [ ]:
description_issue2 = "It would be useful to be able to estimate the amount of repair streaming that needs to be done, without actually doing any streaming. Our main motivation for this having something this is validating CASSANDRA-9143 in production, but I’d imagine it could also be a useful tool in troubleshooting."
print(description_issue2)
print(analyse_issue(issue_field=description_issue2))

It would be useful to be able to estimate the amount of repair streaming that needs to be done, without actually doing any streaming. Our main motivation for this having something this is validating CASSANDRA-9143 in production, but I’d imagine it could also be a useful tool in troubleshooting.
Architectural Impact: Yes


In [ ]:
my_test = """ In Apache Cassandra, a distributed NoSQL database system, data is replicated across multiple nodes for fault tolerance and scalability. However, there can be situations where the replicated data becomes inconsistent across these nodes. This inconsistency can lead to various problems, including:

Stale Reads: A client might read outdated data if it happens to query a node that hasn't received the latest update.
Write Failures: Write operations might fail if enough replicas haven't acknowledged the write, leading to data loss.
Inconsistency between Reads and Writes: A client might read data that is different from what it just wrote due to replication lag.
"""

In [ ]:
print(my_test)
print(analyse_issue(issue_field=my_test))

 In Apache Cassandra, a distributed NoSQL database system, data is replicated across multiple nodes for fault tolerance and scalability. However, there can be situations where the replicated data becomes inconsistent across these nodes. This inconsistency can lead to various problems, including:

Stale Reads: A client might read outdated data if it happens to query a node that hasn't received the latest update.
Write Failures: Write operations might fail if enough replicas haven't acknowledged the write, leading to data loss.
Inconsistency between Reads and Writes: A client might read data that is different from what it just wrote due to replication lag.

Architectural Impact: No


In [ ]:
my_test = "Config upper bound should be handled on startup/config setup and not during conversion"
print(my_test)
print(analyse_issue(issue_field=my_test))

Config upper bound should be handled on startup/config setup and not during conversion
Architectural Impact: Yes


In [ ]:
my_test = """Replacing a dead node with a new one is a common operation, but "nodetool removetoken" followed by bootstrap is inefficient (re-replicating data first to the remaining nodes, then to the new one) and manually bootstrapping to a token "just less than" the old one's, followed by "nodetool removetoken" is slightly painful and prone to manual errors.

First question: how would you expose this in our tool ecosystem?  It needs to be a startup-time option to the new node, so it can't be nodetool, and messing with the config xml definitely takes the "convenience" out.  A one-off -DreplaceToken=XXY argument?
"""

In [ ]:
print(my_test)
print(analyse_issue(issue_field=my_test))

Replacing a dead node with a new one is a common operation, but "nodetool removetoken" followed by bootstrap is inefficient (re-replicating data first to the remaining nodes, then to the new one) and manually bootstrapping to a token "just less than" the old one's, followed by "nodetool removetoken" is slightly painful and prone to manual errors.

First question: how would you expose this in our tool ecosystem?  It needs to be a startup-time option to the new node, so it can't be nodetool, and messing with the config xml definitely takes the "convenience" out.  A one-off -DreplaceToken=XXY argument?

Architectural Impact: No


In [ ]:
my_test = """To support multiple applications on top of a single Cassandra cluster (and to protect against badly behaving clients) having a very simple scheduler for client operations would be very beneficial.

Since all tasks are short lived, a sufficient scheduler would probably only need to manage the queue of incoming requests, and weight them based on an assigned ID. The ID could be dynamically determined by using ip, userid or keyspace for instance, and then each Runnable would be assigned an ID.
"""

In [ ]:
print(my_test)
print(analyse_issue(issue_field=my_test))

To support multiple applications on top of a single Cassandra cluster (and to protect against badly behaving clients) having a very simple scheduler for client operations would be very beneficial.

Since all tasks are short lived, a sufficient scheduler would probably only need to manage the queue of incoming requests, and weight them based on an assigned ID. The ID could be dynamically determined by using ip, userid or keyspace for instance, and then each Runnable would be assigned an ID.

Architectural Impact: Yes
